# Défi Grosses Data 2018
# Algorithme réseaux de neurones avec KERAS

## Préparation des données

In [40]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)

<module 'Annex' from '/home/marques/Documents/Projet_Meteo/Defi/Annex.py'>

In [34]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.1,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='drop')

We are addressing your request.
Data has been imported. Size: (189280, 31)


/home/marques/Documents/Projet_Meteo/Defi/Annex.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='octobre']='10'
/home/marques/Documents/Projet_Meteo/Defi/Annex.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='novembre']='11'
/home/marques/Documents/Projet_Meteo/Defi/Annex.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='décembre']='12'
/home/marques/Documents/Projet_Meteo/Defi/Annex.py:149: SettingWithCopyWarning: 
A 

Months converted to int.
Dummies added.
Date dropped.
26528 data points deleted. 14.02 %
Train size: 146476, Test size: 16276
Data scaled


#### Jeu de données imputé

Mise au format numérique et centré

In [23]:
X_train_imp,X_test_imp,Y_train_imp,Y_test_imp,scaler_imp=Annex.get_data_raw(scale=False, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.1,
                                                        impute_method='imputed',
                                                        convert_month2int=True,
                                                        date_method='drop')

X_train_imp.apply(pd.to_numeric, axis=1)
X_test_imp.apply(pd.to_numeric, axis=1)

scaler = StandardScaler()  
scaler.fit(X_train_imp)  
X_train_imp = scaler.transform(X_train_imp)  
X_test_imp = scaler.transform(X_test_imp)

We are addressing your request.
Data has been imported. Size: (189280, 85)
Train size: 170352, Test size: 18928


NameError: name 'StandardScaler' is not defined

## Réseau de neurones avec Keras


In [25]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,SimpleRNN, Lambda,GRU,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,LSTM,LocallyConnected2D,Convolution2D,Reshape,Conv1D
from keras.utils import np_utils
from keras.optimizers import RMSprop,Nadam,Adam,SGD
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import time
from sklearn.model_selection import GridSearchCV

print(keras.__version__)

2.0.2


#### Définition des séquences du modèle

In [26]:
def baseline_model():
    # create model
    model = Sequential()
    #model.add(Conv1D(50,input_dim=38, kernel_size=3))
    model.add(Dense(80, input_dim=38, kernel_initializer='normal', activation='relu'))
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [27]:
X_train.shape

(146476, 38)

In [47]:
 
def baseline_model():  
    # create model  
    model = Sequential()  
    #model.add(Conv1D(filters=10,kernel_size=3,border_mode='valid', input_shape=(None,38)))
    model.add(Dense(200, input_shape=(38,),kernel_initializer='normal', activation='relu'))
    #model.add(Conv1D(50,kernel_size=3,strides=1))
    model.add(SimpleRNN(units=30))
    model.add(Dense(200, kernel_initializer='normal', activation='relu'))  
    model.add(Dense(1, kernel_initializer='normal'))  
    # Compile model  
    model.compile(loss='mean_squared_error', optimizer='adam')  
    return model   


#### Modèles qui ont donné de bons résultats (1.17) 

Ce modèle là, quitte à ajouter une couche dense de 100 neurones.
Attention, en fonction du jeu de données de départ (imputé ou pas), régler la taille de la couche d'entrée. 

In [29]:

def baseline_model():    
    # create model  
    model = Sequential()  
    model.add(Dense(200, input_shape=(38,), kernel_initializer='normal', activation='relu'))  
    model.add(Dense(200, kernel_initializer='normal', activation='relu'),) 
    model.add(Activation("relu"))
    model.add(Dense(1, kernel_initializer='normal'))  
    # Compile model  
    model.compile(loss='mean_squared_error', optimizer='adam')  
    return model  
    

#### Construction du modèle et estimation sur échantillon test (pas de validation croisée pour l'instant)

In [ ]:
X_train_conv=np.expand_dims(X_train, axis=0)
X_train_conv.shape

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset

nb_epoch=30
batch_size=15

ts = time.time()
nnet = KerasRegressor(build_fn=baseline_model, epochs=nb_epoch, batch_size=batch_size, verbose=1)

nnet.fit(X_train,Y_train)
score = nnet.score(X_test, Y_test)
ypred = nnet.predict(X_test)
te = time.time()

In [20]:
print("Score : %f, time running : %d secondes" %(score, te-ts))

Score : 1.011370, time running : 497 secondes


#### Validation croisée sur les paramètres du réseau de neurones

Optimisation du nombre d'époques

##### Question : pourquoi le nombre d'époque est toujours à 10 ? (Même si on change le paramètre ?)

A FAIRE : 
- changer l'optimizer (dans baseline_model1 : tester sgd 
##### Moins bon que adam
- tester la convolution en 1D pour le temps (pour cela, rajouter une ou plusieurs variables sur la date)
- optimiser la taille du batch (augmenter ?)
- optimiser si possible les paramètres du baseline_model (ex : elu au lieu de relu pour fonction d'activation)
- tester avec echantillons avec complétion de NAs
- tester cross validation

In [ ]:
ts = time.time()
param_grid=[{"batch_size":[25,40]}]
nnet=GridSearchCV(KerasRegressor(build_fn=baseline_model, verbose=1),param_grid,cv=5,n_jobs=-1)
nnet.fit(X_train,Y_train, epochs=1)
score = nnet.score(X_test, Y_test)
ypred = nnet.predict(X_test)
te = time.time()
t_total = te-ts

In [ ]:
nnet.best_params_["nb_epoch"]
print("Meilleur score = %f, Meilleur paramètre = %s" % (nnet.best_score_,nnet.best_params_))

Autres essais 

In [42]:
kfold = KFold(n_splits=5, random_state=seed)

ts = time.time()
results = cross_val_score(nnet, X_train, Y_train, cv=kfold)
te = time.time()

print("Results: %.2f (%.2f) MSE ; time running : %d secondes" % (results.mean(), results.std(),te-ts))

Epoch 1/30
117180/117180 [==============================] - 9s - loss: 2.3663     
Epoch 2/30
117180/117180 [==============================] - 9s - loss: 1.4698     
Epoch 3/30
117180/117180 [==============================] - 11s - loss: 1.3754    
Epoch 4/30
117180/117180 [==============================] - 11s - loss: 1.3098    
Epoch 5/30
117180/117180 [==============================] - 12s - loss: 1.2721    
Epoch 6/30
117180/117180 [==============================] - 11s - loss: 1.2337    
Epoch 7/30
117180/117180 [==============================] - 12s - loss: 1.2123    
Epoch 8/30
117180/117180 [==============================] - 11s - loss: 1.1909    
Epoch 9/30
117180/117180 [==============================] - 11s - loss: 1.1712    
Epoch 10/30
117180/117180 [==============================] - 12s - loss: 1.1469    
Epoch 11/30
117180/117180 [==============================] - 12s - loss: 1.1333    
Epoch 12/30
117180/117180 [==============================] - 12s - loss: 1.1189    
E

117181/117181 [==============================] - 13s - loss: 1.2334    
Epoch 7/30
117181/117181 [==============================] - 13s - loss: 1.2047    
Epoch 8/30
117181/117181 [==============================] - 13s - loss: 1.1847    
Epoch 9/30
117181/117181 [==============================] - 12s - loss: 1.1646    
Epoch 10/30
117181/117181 [==============================] - 11s - loss: 1.1412    
Epoch 11/30
117181/117181 [==============================] - 11s - loss: 1.1217    
Epoch 12/30
117181/117181 [==============================] - 11s - loss: 1.1072    
Epoch 13/30
117181/117181 [==============================] - 14s - loss: 1.0898    
Epoch 14/30
117181/117181 [==============================] - 17s - loss: 1.0790    
Epoch 15/30
117181/117181 [==============================] - 14s - loss: 1.0658    
Epoch 16/30
117181/117181 [==============================] - 15s - loss: 1.0524    
Epoch 17/30
117181/117181 [==============================] - 12s - loss: 1.0384    
Epoch 1

#### Generate submission file

In [41]:
Annex.generate_submission_file("submission_13dec2017_15h00_rnnkeras.csv", nnet, scaler, add_dummies=True, var_dummies=['insee','ddH10_rose4'], 
                         convert_month2int=True, date_method='drop', fillna_method='zeros' )

Months converted to int.
Dummies added.
Date dropped.
20595/21168 [============================>.] - ETA: 0s

'File ./../data_meteo/submission_13dec2017_15h00_rnnkeras.csv generated.'